In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from typing import Union, Literal, Callable, Tuple, List
import warnings

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    label_from_attrs,
    adjust_lightness_array,
    plot_one_one,
    handler_map_alpha,
    save_figure,
    add_second_x_axis,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean
from sdm_eurec4a import RepositoryPath

from sdm_eurec4a import slurm_cluster

default_colors = set_custom_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")

data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.1")


fig_dir = RepoPaths.fig_dir / Path("paper-v4.1")
fig_dir.mkdir(exist_ok=True, parents=False)

In [ ]:
client, cluster = slurm_cluster.init_dask_slurm_cluster(
    scale=1, processes=32, walltime="00:45:00", memory="32GB"
)

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /scratch/m/m301096/dask_logs/dask-worker-%J.err
#SBATCH -o /scratch/m/m301096/dask_logs/dask-worker-%J.out
#SBATCH -p compute
#SBATCH -A um1487
#SBATCH -n 1
#SBATCH --cpus-per-task=32
#SBATCH --mem=30G
#SBATCH -t 00:45:00

/work/um1487/m301096/conda/envs/sdm_pysd_python312/bin/python -m distributed.cli.dask_worker tcp://10.128.4.106:39851 --name dummy-name --nthreads 1 --memory-limit 0.93GiB --nworkers 32 --nanny --death-timeout 60 --local-directory /scratch/m/m301096/dask_temp --interface ib0



In [ ]:
eulerian_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/eulerian_dataset_combined.nc"
)
conservation_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/conservation_dataset_combined.nc"
)
mean_radius_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/mean_radius_combined.nc"
)

In [ ]:
microphysics = (
    "null_microphysics",
    "condensation",
    # "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

compare_microphysics = (
    # "null_microphysics",
    "condensation",
    # "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

labels = {
    "null_microphysics": "Null",
    "condensation": "Evap only",
    # "collision_condensation" : "Coll",
    "coalbure_condensation_small": "CoalBuRe few",
    "coalbure_condensation_large": "CoalBuRe many",
}

colors = {
    "null_microphysics": "grey",
    "condensation": "purple",
    # "collision_condensation" : "blue",
    "coalbure_condensation_small": "red",
    "coalbure_condensation_large": "orange",
}

dark_colors: dict = dict(zip(colors.keys(), adjust_lightness_array(colors.values(), 0.75)))
light_colors: dict = dict(zip(colors.keys(), adjust_lightness_array(colors.values(), 1.25)))
markers = {
    "null_microphysics": "$*$",
    "condensation": "1",
    # "collision_condensation" :"x",
    "coalbure_condensation_small": "+",
    "coalbure_condensation_large": "2",
}

In [ ]:
plt.hist(ds["evaporation_fraction"].sel(microphysics="condensation") * 1e-18)

(array([59., 23., 21.,  9.,  2.,  1.,  2.,  0.,  0.,  7.]),
 array([ 0.37695808,  7.97889138, 15.58082469, 23.18275799, 30.78469129,
        38.38662459, 45.98855789, 53.5904912 , 61.1924245 , 68.7943578 ,
        76.3962911 ]),
 <BarContainer object of 10 artists>)

## Data loading

In [ ]:
time_slice = slice(1500, 3490)
chunks = dict(cloud_id=2)

l = []
for microphysic in microphysics:
    print(microphysic)
    ds_euler_org = xr.open_dataset(eulerian_data_path(microphysic), chunks=chunks)
    ds_euler = ds_euler_org[
        [
            "massdelta_condensation",
            "gridbox_volume",
            "gridbox_coord3",
            "relative_humidity",
            "max_gridbox",
            "liquid_water_content",
            "mass_moment1",
        ]
    ]
    ds_euler = ds_euler.sel(time=time_slice).mean("time", keep_attrs=True)
    ds_euler["surface_area"] = ds_euler_org["surface_area"].mean()
    ds_euler["surface_area"].attrs["units"] = "m^2"
    ds_euler["surface_area"].attrs["long_name"] = "Surface area"

    ds_euler["max_gridbox"] = ds_euler["max_gridbox"].fillna(ds_euler["gridbox"].max())

    ds_euler["total_precipitation"] = (
        ds_euler_org["precipitation"]
        .sum("radius_bins", keep_attrs=True)
        .sel(time=time_slice)
        .mean("time", keep_attrs=True, skipna=False)
    )

    # from kg per dT to g per h per m^2
    ds_conser = xr.open_dataset(conservation_data_path(microphysic), chunks=chunks)
    ds_conser = ds_conser.sel(time=time_slice).mean("time", keep_attrs=True)

    ds_mean_radius = xr.open_dataset(mean_radius_data_path(microphysic), chunks=chunks)

    ds_single = xr.merge([ds_conser, ds_euler, ds_mean_radius])
    ds_single = ds_single.expand_dims(microphysics=(microphysic,))
    l.append(ds_single)
ds = xr.concat(l, dim="microphysics")
ds.chunk(chunks)

ds["max_gridbox"] = ds["max_gridbox"].fillna(ds["gridbox"].max())

ds["liquid_water_content"] = 1e3 * ds["liquid_water_content"]
ds["liquid_water_content"].attrs["units"] = "g/m^3"
ds["liquid_water_content"].attrs["long_name"] = "Liquid water content"

ds["cloud_liquid_water_content"] = ds["liquid_water_content"].sel(gridbox=ds["max_gridbox"])
ds["cloud_liquid_water_content"].attrs["long_name"] = "Cloud liquid water content"

ds["cloud_xi_radius_mean"] = ds["xi_radius_mean"].sel(gridbox=ds["max_gridbox"])
ds["cloud_xi_radius_mean"].attrs["long_name"] = "Cloud mean radius"

ds["small_cloud_xi_radius_mean"] = ds["small_xi_radius_mean"].sel(gridbox=ds["max_gridbox"])
ds["small_cloud_xi_radius_mean"].attrs["long_name"] = "Cloud mean radius"


ds["cloud_mass_radius_mean"] = ds["mass_radius_mean"].sel(gridbox=ds["max_gridbox"])
ds["cloud_mass_radius_mean"].attrs["long_name"] = "Cloud mean mass radius"

ds["small_cloud_mass_radius_mean"] = ds["small_mass_radius_mean"].sel(gridbox=ds["max_gridbox"])
ds["small_cloud_mass_radius_mean"].attrs["long_name"] = "Cloud mean mass radius"


ds["source"].attrs["long_name"] = "Evaporation"
ds["inflow"].attrs["long_name"] = "Cloud Base Precipitation Flux"
ds["outflow"].attrs["long_name"] = "Surface Precipitation Flux"

# from kg/m^3/s

# # to mg/m^3/h
# ds['evaporation_rate']  = - ds['massdelta_condensation'] * 1e6 * 3600
# ds['evaporation_rate'].attrs['units'] = 'mg/m^3/h'
# ds['evaporation_rate'].attrs['long_name'] = 'Evaporation rate'

# to mm/m/h
rho_water = 1000  # kg / m^3
ds["evaporation_rate"] = -1e3 / rho_water * ds["massdelta_condensation"] * 3600
ds["evaporation_rate"].attrs["units"] = r"mm \, h^{-1} \, m^{-1}"
ds["evaporation_rate"].attrs["long_name"] = "Evaporation rate"

# from kg / m^3 / s
# to W / m^3
vapourization_heat = 2257e3  # J / kg
rho_water = 1000  # kg / m^3
ds["evaporation_rate_energy"] = ds["massdelta_condensation"] * vapourization_heat  # J/s / m^3 = W / m^3
ds["evaporation_rate_energy"] = 1e3 * ds["evaporation_rate_energy"]  # mW / m^3
ds["evaporation_rate_energy"].attrs["units"] = r"mW \, m^{-3}"
ds["evaporation_rate_energy"].attrs["long_name"] = "Evaporation rate"


for var, new_var in zip(
    ["source", "inflow", "outflow", "reservoir_change"],
    [
        "source_precipitation",
        "inflow_precipitation",
        "outflow_precipitation",
        "reservoir_change_precipitation",
    ],
):
    attrs = ds[var].attrs.copy()
    # from  kg per dT per domain area
    # dT = 2s

    # # to    g per h per m^2
    # ds[var] = ds[var] / 2 * 3600 / ds['surface_area'] * 1e6
    # ds[var].attrs.update(attrs)
    # ds[var].attrs['units'] = 'mg/m^2/h'

    # to    mm / h
    rho_water = 1000  # kg / m^3
    ds[new_var] = ds[var] / 2 * 3600 / ds["surface_area"]  # kg / m^2 / h
    ds[new_var] = 1e3 * ds[new_var] / rho_water  # mm / h
    ds[new_var].attrs.update(attrs)
    ds[new_var].attrs["units"] = r"mm \, h^{-1}"

for var, new_var in zip(
    ["source", "inflow", "outflow", "reservoir_change"],
    ["source_energy", "inflow_energy", "outflow_energy", "reservoir_change_energy"],
):
    attrs = ds[var].attrs.copy()
    # from  kg per dT per domain area
    # dT = 2s

    # # to    g per h per m^2
    # ds[var] = ds[var] / 2 * 3600 / ds['surface_area'] * 1e6
    # ds[var].attrs.update(attrs)
    # ds[var].attrs['units'] = 'mg/m^2/h'

    # to    mm / h
    vapourization_heat = 2257e3  # J / kg
    rho_water = 1000  # kg / m^3
    ds[new_var] = ds[var] / 2 / ds["surface_area"]  # kg / m^2 / s
    ds[new_var] = ds[new_var] * vapourization_heat  # J/s = W / m^2
    ds[new_var].attrs.update(attrs)
    ds[new_var].attrs["units"] = r"W \, m^{-2}"


ds["evaporation_fraction"] = -100 * ds["source"] / ds["inflow"]
ds["evaporation_fraction"].attrs["units"] = "\\%"
ds["evaporation_fraction"].attrs["long_name"] = "Evaporation fraction"

sorted_cloud_id = ds["cloud_id"].sortby(ds["evaporation_rate"].max(["gridbox", "microphysics"]))
ds_backup = ds
# ds = ds.sel(cloud_id = sorted_cloud_id[:-2])
# ds_diff = ds - ds.sel(microphysics = 'condensation')

null_microphysics
condensation
coalbure_condensation_small
coalbure_condensation_large


In [ ]:
ds = ds_backup.load()  # .sel(cloud_id = sorted_cloud_id[:-2])
ds = ds.sel(cloud_id=sorted_cloud_id[:-2])
ds = ds.sortby("cloud_id")
ds_diff = ds - ds.sel(microphysics="condensation")


for var in ds_diff.data_vars:
    ds_diff[var].attrs.update(ds[var].attrs)
    ds_diff[var].attrs["long_name"] = f"Difference {ds[var].attrs['long_name']}"

In [ ]:
def evaporation_mass2evaporation_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    L_v = 2.2257e3

    return x * L_v


def evaporation_mass2evaporation_precipitation(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    # L_v = 2.2257e3

    return x * 3600 / 1000 * rho_water


def evaporation_precipitation2evaporation_mass(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    L_v = 2257e3  # J / kg
    return x / 3600 * 1000 / rho_water


def evaporation_precipitation2evaporation_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    L_v = 2257e3  # J / kg

    # x in mm/h

    # output in W/m^2

    x = x / 1000 * rho_water / 3600  # kg/m^2/s

    return x * L_v  # W/m^2


def evap_rate_mass2evap_rate_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:

    L_v = 2.2257e3

    # x in mW m^-3
    return x / L_v  # g/m^3/s

# Visulization

## Precipitation

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in microphysics:

    ax.scatter(
        -ds.sel(microphysics=mp)["outflow_precipitation"],
        ds.sel(microphysics=mp)["total_precipitation"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )


ax.plot(
    -ds["outflow_precipitation"],
    ds["total_precipitation"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")

ax.set_xlabel("Calculated using the Box model attempt\n" + label_from_attrs(ds["outflow_precipitation"]))
ax.set_ylabel("Calculated using the Eulerian attempt\n" + label_from_attrs(ds["total_precipitation"]))
plot_one_one(ax, color="black", linestyle="--")

save_figure(fig, fig_dir, Path("precipitation_box_vs_eulerian"))

### Precipitation statistics

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = -ds["outflow_precipitation"]

for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )
    # ax.fill_betweenx(
    #     [0, 1],
    #     m - sem,
    #     m + sem,
    #     color=colors[mp],
    #     alpha=0.1,
    # )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 10, 0.2),
        label=label,
        # density=True,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 10, 0.2),
        histtype="step",
        # density=True,
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")
save_figure(fig, fig_dir, f"histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = -ds["outflow_energy"]

for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )
    # ax.fill_betweenx(
    #     [0, 1],
    #     m - sem,
    #     m + sem,
    #     color=colors[mp],
    #     alpha=0.1,
    # )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=20,
        # bins=np.arange(0, 10, 0.2),
        label=label,
        # density=True,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=20,
        # bins=np.arange(0, 10, 0.2),
        histtype="step",
        # density=True,
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")
save_figure(fig, fig_dir, f"histogram_{x.name}")

## Evaporation only setup

### Evaporation profiles

In [ ]:
mp = "condensation"

x = -ds["evaporation_rate_energy"]
y = ds["gridbox_coord3"]

fig, ax = plt.subplots()
ax.plot(
    x.sel(microphysics=mp).T,
    y.sel(microphysics=mp).T,
    color=colors[mp],
    alpha=0.1,
)

# plt.xscale('log')
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"{x.name}_vs_{y.name}_{mp}")

In [ ]:
y_ticks = np.arange(0, 1300, 250)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        compare_microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

for mp in compare_microphysics:

    md = ds.sel(microphysics=mp)["evaporation_rate_energy"]
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    z = ds.sel(microphysics=mp)["gridbox_coord3"]
    z_mean, z_std = mean_and_stderror_of_mean(
        data=z,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        z.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    ax.plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        z_mean,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=colors[mp],
    )
ax.set_yticks(y_ticks, y_ticks)
ax.legend()
ax.set_xlim(-150, 0)
ax0.set_xlim(-150, 0)

fig.supxlabel(label_from_attrs(ds["evaporation_rate_energy"]))
fig.supylabel("Height [m]")
fig.suptitle("Evaporation profiles")
fig.tight_layout()
save_figure(fig, fig_dir, "evaporation_profiles")

In [ ]:
y_ticks = np.arange(0, 1300, 250)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        compare_microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

for mp in compare_microphysics:

    md = ds_diff.sel(microphysics=mp)["evaporation_rate_energy"]
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    z = ds.sel(microphysics=mp)["gridbox_coord3"]
    z_mean, z_std = mean_and_stderror_of_mean(
        data=z,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        z.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])
    # axs[mp].set_xscale('symlog', linthresh = 1e-8)
    ax.plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        z_mean,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.3,
        color=colors[mp],
    )


ax.set_yticks(y_ticks, y_ticks)

ax.legend(loc="upper right")
ax.set_xlim(-15, 15)
ax0.set_xlim(-30, 30)


fig.supxlabel(label_from_attrs(ds["evaporation_rate_energy"]))
fig.supylabel("Height [m]")
fig.suptitle("Evpoartion profiles | Difference to Evaporation only setup ")
fig.tight_layout()

save_figure(fig, fig_dir, "evaporation_profiles_diff")

In [ ]:
plot_microphysics = [
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
]

N = len(plot_microphysics) + 1
width_ratios = np.ones(N)
width_ratios[-1] = 0.05

fig, axs = plt.subplots(nrows=1, ncols=N, figsize=np.array((12 * 3, 9)) * 0.5, width_ratios=width_ratios)

cax = axs[-1]
axs_flat = axs[:-1].flatten()

x = ds["liquid_water_content"]
y = ds["relative_humidity"]
color_data = -ds_diff["evaporation_rate_energy"]

cmap = strength_cmap
cmap = sns.diverging_palette(h_neg=250, h_pos=30, s=100, l=35, center="light", as_cmap=True)
# cmap = cmap.resampled(10)

norm = mcolors.SymLogNorm(
    linthresh=0.1,
    linscale=0.5,
    vmin=-55,
    vmax=55,
)
# norm = None

for i, mp in enumerate(plot_microphysics):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=20,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_title(labels[mp])


fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
fig.colorbar(
    mappable=sc,
    cax=cax,
    label=label_from_attrs(color_data),
    # ticks = np.arange(-100, 101, 20),
)
fig.tight_layout()
save_figure(fig, fig_dir, f"all-diff-{x.name}-{y.name}-color-{color_data.name}")

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=np.array((16, 9)) * 0.5)

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = -ds["evaporation_rate_energy"]

# color_data = color_data / color_data.sel(gridbox = 0) * 100
y = ds["relative_humidity"]
x = ds["liquid_water_content"]


cmap = "inferno_r"


norm = mcolors.Normalize(
    vmin=0,
    vmax=200,
)

for i, mp in enumerate(plot_microphysics):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=30,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])

cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)

save_figure(fig, fig_dir, f"{x.name}-{y.name}-color-{color_data.name}")

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=np.array((16, 9)) * 0.5)

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = -ds["evaporation_rate_energy"]

color_data = (color_data / color_data.mean("gridbox") - 1) * 100
color_data.attrs["units"] = "\\%"
color_data.attrs["long_name"] = "Evaporation rate relative to column mean"
color_data.name = "evaporation_rate_energy_relative"

y = ds["relative_humidity"]
x = ds["liquid_water_content"]

cmap = "RdBu_r"
cmap = sns.diverging_palette(h_neg=250, h_pos=30, s=100, l=50, center="light", as_cmap=True, n=3)
cmap = cmap.resampled(10)

norm = mcolors.Normalize(
    vmin=-100,
    vmax=100,
)

for i, mp in enumerate(plot_microphysics):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=20,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(
    mappable=sc,
    cax=cax,
    label=label_from_attrs(color_data),
    ticks=np.arange(-100, 101, 20),
)
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
save_figure(fig, fig_dir, f"{x.name}-{y.name}-color-{color_data.name}")

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=np.array((16, 9)) * 0.5)

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

x = ds["liquid_water_content"]
y = ds["mass_radius_mean"]
color_data = -ds["evaporation_rate_energy"]

# color_data = (color_data / color_data.mean('gridbox') - 1) * 100
# color_data.attrs["units"] = "\\%"
# color_data.attrs["long_name"] = "Evaporation rate relative to column mean"
# color_data.name = "evaporation_rate_energy_relative"

cmap = strength_cmap
# cmap = sns.diverging_palette(h_neg= 250, h_pos= 30, s= 100, l= 50, center = 'light', as_cmap=True, n = 3)
# cmap = cmap.resampled(10)

norm = mcolors.Normalize(
    vmin=0,
    vmax=150,
)

for i, mp in enumerate(plot_microphysics):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=20,
        alpha=0.8,
        # label = labels[mp],
    )
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(
    mappable=sc,
    cax=cax,
    label=label_from_attrs(color_data),
    # ticks = np.arange(-100, 101, 20),
)
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
save_figure(fig, fig_dir, f"{x.name}-{y.name}-color-{color_data.name}")

### Evaporation histograms

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["evaporation_fraction"]
for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 100, 5),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 100, 5),
        histtype="step",
    )


ax.axvline(100, color="black", linestyle="--")

ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")

save_figure(fig, fig_dir, f"histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = -ds["source_precipitation"]
for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 0.65, 0.025),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 0.65, 0.025),
        histtype="step",
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


add_second_x_axis(
    ax,
    new_xticks_func=lambda x: evaporation_precipitation2evaporation_energy(x).astype(int),
    xlabel=label_from_attrs(ds["source_energy"]),
    position="bottom",
    offset_position=("axes", -0.15),
)


# add_second_x_axis(
#     ax,
#     new_xticks_func=lambda x : (3600 * 1e3 * evaporation_precipitation2evaporation_mass(x)).astype(int),
#     xlabel="Evaporation rate [kg/m^2/h]",
# position = ("axes", -0.30)
# )

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")
fig.tight_layout()

save_figure(fig, fig_dir, f"histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = -ds["source_precipitation"]
for mp in compare_microphysics:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 0.65, 0.025),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 0.65, 0.025),
        histtype="step",
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


add_second_x_axis(
    ax,
    new_xticks_func=lambda x: evaporation_precipitation2evaporation_energy(x).astype(int),
    xlabel=label_from_attrs(ds["source_energy"]),
    position="bottom",
    offset_position=("axes", -0.15),
)


# add_second_x_axis(
#     ax,
#     new_xticks_func=lambda x : (3600 * 1e3 * evaporation_precipitation2evaporation_mass(x)).astype(int),
#     xlabel="Evaporation rate [kg/m^2/h]",
# position = ("axes", -0.30)
# )

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")
fig.tight_layout()

save_figure(fig, fig_dir, f"all-histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["evaporation_fraction"]
for mp in ds["microphysics"].values:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 100, 5),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 100, 5),
        histtype="step",
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(ds["evaporation_fraction"]))
ax.set_ylabel("Counts")

save_figure(fig, fig_dir, f"all_histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow_precipitation"],
        ds.sel(microphysics=mp)["evaporation_fraction"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")


ax.set_ylabel(label_from_attrs(ds["evaporation_fraction"]))
ax.set_xlabel(label_from_attrs(ds["inflow_precipitation"]))

Text(0.5, 0, 'Cloud Base Precipitation Flux $\\left[  mm \\, h^{-1}  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds["cloud_liquid_water_content"].sel(microphysics=mp),
        ds["evaporation_fraction"].sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")


ax.set_ylabel(label_from_attrs(ds["evaporation_fraction"]))
ax.set_xlabel(label_from_attrs(ds["cloud_liquid_water_content"]))

Text(0.5, 0, 'Cloud liquid water content $\\left[  g/m^3  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds["cloud_liquid_water_content"].sel(microphysics=mp),
        -ds["source_energy"].sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
ax.set_yscale("log")
ax.set_xscale("log")


ax.set_ylabel(label_from_attrs(ds["source_energy"]))
ax.set_xlabel(label_from_attrs(ds["cloud_liquid_water_content"]))

Text(0.5, 0, 'Cloud liquid water content $\\left[  g/m^3  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

plot_microphysics = ["condensation"]

for mp in plot_microphysics:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow_precipitation"],
        -ds.sel(microphysics=mp)["source_precipitation"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    ds.sel(microphysics=plot_microphysics)["inflow_precipitation"],
    -ds.sel(microphysics=plot_microphysics)["source_precipitation"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    x = 15
    y = p * x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )
    y = 2e-4
    x = y / p
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(ds["inflow_precipitation"]))
ax.set_ylabel(label_from_attrs(ds["source_precipitation"]))

Text(0, 0.5, 'Evaporation $\\left[  mm \\, h^{-1}  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)
ax = fig.add_subplot(gs[:, :])
plot_microphysics = ["condensation"]
x = (ds["inflow_precipitation"],)
y = (-ds["source_precipitation"],)

for mp in plot_microphysics:
    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )
    ax.plot(
        x.sel(microphysics=plot_microphysics),
        -ds.sel(microphysics=plot_microphysics),
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )
    ax.legend(loc="upper left")
    ax.set_xscale("log")
    ax.set_yscale("log")
    # ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
    lims = np.array([1e-6, 2.5e1])
    ax.set_ylim(lims.min(), lims.max())
    ax.set_xlim(1e-4, lims.max())
    p_x_values = np.geomspace(lims.min(), lims.max(), 100)
    for p in [1, 0.1, 0.01]:
        style = dict(color="grey", alpha=p ** (1 / 5))
        lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
        line = lines[0]
        annotate_x = 15
        annotate_y = p * annotate_x
        # y = 1e-0
        # x =(1/1.3) * (y / p)
        ax.annotate(
            f"{100 * p:.0f} %",
            xy=(annotate_x, annotate_y),
            xytext=(0, 0),
            textcoords="offset points",
            va="top",
            ha="left",
            size=10,
            **style,
        )
        annotate_y = 2e-4
        annotate_x = annotate_y / p
        ax.annotate(
            f"{100 * p:.0f} %",
            xy=(annotate_x, annotate_y),
            xytext=(0, 0),
            textcoords="offset points",
            va="top",
            ha="left",
            size=10,
            **style,
        )
    ax.set_xlabel(label_from_attrs(x))
    ax.set_ylabel(label_from_attrs(y))

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in microphysics[1:]:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow_precipitation"],
        -ds.sel(microphysics=mp)["source_precipitation"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    ds.sel(microphysics=list(microphysics[1:]))["inflow_precipitation"],
    -ds.sel(microphysics=list(microphysics[1:]))["source_precipitation"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 9])
ax.set_ylim(0, 1)
ax.set_xlim(0, 9)

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    x = 7e0
    y = p * x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="center",
        ha="right",
        size=10,
        **style,
    )
    y = 1e-4
    x = 1.5 * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="center",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(ds["inflow_precipitation"]))
ax.set_ylabel(label_from_attrs(ds["source_precipitation"]))

Text(0, 0.5, 'Evaporation $\\left[  mm \\, h^{-1}  \\right]$')

In [ ]:
fig = plt.figure()

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

for mp in microphysics:
    for _ax in axs:
        _ax.scatter(
            ds.sel(microphysics=mp)["inflow_precipitation"],
            ds.sel(microphysics=mp)["cloud_liquid_water_content"],
            color=colors[mp],
            alpha=0.5,
            marker=markers[mp],
            label=labels[mp],
        )
for _ax in axs:
    _ax.plot(
        ds["inflow_precipitation"],
        ds["cloud_liquid_water_content"],
        color="grey",
        linewidth=0.2,
        alpha=0.2,
        zorder=0,
    )

    _ax.legend(loc="upper left")
axs[1].set_yscale("log")
axs[1].set_xscale("log")

for _ax in axs:

    _ax.set_xlabel(label_from_attrs(ds["inflow_precipitation"]))
    _ax.set_ylabel(label_from_attrs(ds["cloud_liquid_water_content"]))
fig.suptitle("Relation of Inflow and Cloud Liquid Water Content")
fig.tight_layout()

<Figure size 800x450 with 0 Axes>

### Impoact of Rain water content

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = -ds["source_energy"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"{x.name}_vs_{y.name}")

### Impact of relative humidity

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = (
    (ds["relative_humidity"].max("gridbox") - ds["relative_humidity"].min("gridbox"))
    / ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"])
    * 1e3
)
x.name = "RH_gradient"
x.attrs["units"] = r"\% km$^{-1}$"
x.attrs["long_name"] = "Relative Humidity Gradient"
y = -ds["source_energy"]  # / ds["gridbox_coord3"].sel(gridbox = ds['max_gridbox']) * 1e3

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"{x.name}_vs_{y.name}")

### Correlation of evaporation rate against the rest

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

dss = ds.sel(microphysics="condensation")

bins = np.arange(-1, 1.01, 0.05)

x = xr.corr(dss["liquid_water_content"], -dss["evaporation_rate_energy"], dim="gridbox")
x.attrs["units"] = r"\,"
x.attrs["long_name"] = "RWC Corr"
x.name = "lwc_correlation_coefficient"
m, sem = mean_and_stderror_of_mean(
    data=x,
    dims="cloud_id",
)

label = (
    label_from_attrs(x, return_units=False) + f': {m.values:.2f} ± {sem.values:.2f} ${x.attrs["units"]}$'
)

ax.axvline(
    m,
    color=colors[mp],
    linestyle="-",
)

ax.hist(
    x,
    color=colors[mp],
    alpha=0.1,
    bins=bins,
    label=label,
)

ax.hist(
    x,
    color=colors[mp],
    alpha=1,
    linewidth=1,
    bins=bins,
    histtype="step",
)

x = xr.corr(dss["relative_humidity"], -dss["evaporation_rate_energy"], dim="gridbox")
x.attrs["units"] = r"\,"
x.attrs["long_name"] = "RH Corr"
x.name = "rh_correlation_coefficient"
m, sem = mean_and_stderror_of_mean(
    data=x,
    dims="cloud_id",
)

label = (
    label_from_attrs(x, return_units=False) + f': {m.values:.2f} ± {sem.values:.2f} ${x.attrs["units"]}$'
)

ax.axvline(
    m,
    color="orange",
    linestyle="-",
)

ax.hist(
    x,
    color="orange",
    alpha=0.1,
    bins=bins,
    label=label,
)

ax.hist(
    x,
    color="orange",
    alpha=1,
    linewidth=1,
    bins=bins,
    histtype="step",
)

ax.axvline(
    xr.corr(ds["cloud_liquid_water_content"], -ds["source_energy"]),
    color="k",
    linestyle="-",
    label="Column int. correlation",
)


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")

Text(0, 0.5, 'Counts')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = -ds["source_energy"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"{x.name}_vs_{y.name}")

In [ ]:
ds

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

a = (ds["relative_humidity"] / ds["relative_humidity"].mean("gridbox")) - 1

x = (a.max("gridbox") - a.min("gridbox")) / ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"]) * 1e3
x.name = "relative_humidity_gradient"
x.attrs["units"] = r"\% km$^{-1}$"
x.attrs["long_name"] = "Relative humidity gradient"
a = (ds["evaporation_rate_energy"] / ds["evaporation_rate_energy"].mean("gridbox")) - 1

y = (a.max("gridbox") - a.min("gridbox")) / ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"]) * 1e3
y.name = "source_energy_gradient"
y.attrs["units"] = r"mW m$^{-3}$ km$^{-1}$"
y.attrs["long_name"] = "Evaporation rate gradient"

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"{x.name}_vs_{y.name}")

## Impact of mean mass radius

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

test = np.geomspace(6e1, 2e3, 100)
ax.plot(test, 3.7e7 * test ** (-5 / 2), color="k", linestyle="--", label=r"$r^{-5/2}$")

ax.legend(loc="upper right")
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_ylim(None, 100)
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"fit-log-{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

test = np.geomspace(6e1, 2e3, 100)
ax.plot(test, 3.7e7 * test ** (-5 / 2), color="k", linestyle="--", label=r"$r^{-5/2}$")

ax.legend(loc="upper right")
# ax.set_yscale("log")
# ax.set_xscale("log")
ax.set_ylim(0, 100)
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"fit-linear_{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

test = np.geomspace(6e1, 2e3, 100)
ax.plot(test, 3.7e7 * test ** (-5 / 2), color="k", linestyle="--", label=r"$r^{-5/2}$")

ax.legend(loc="upper right")
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_ylim(None, 100)
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"all-fit-log_{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

save_figure(fig, fig_dir, f"all_{x.name}_vs_{y.name}")

## Combined influece RWC and MMR

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=(7, 5))

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = -ds["source_energy"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
y = ds["cloud_mass_radius_mean"]
x = ds["cloud_liquid_water_content"]

color_data = color_data.sel(microphysics=plot_microphysics)
x = x.sel(microphysics=plot_microphysics)
y = y.sel(microphysics=plot_microphysics)


cmap = strength_cmap


norm = mcolors.Normalize(
    vmin=color_data.compute().quantile(0.1).values, vmax=color_data.compute().quantile(0.9).values
)

for i, mp in enumerate(plot_microphysics):

    # cmap = sns.cubehelix_palette(start=i * 0.2, rot=-0.5, as_cmap=True)

    c = color_data.sel(microphysics=mp)
    ax = axs_flat[i]

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=70,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
save_figure(fig, fig_dir, f"{x.name}-{y.name}-color-{color_data.name}")

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=(7, 5))

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = ds["evaporation_fraction"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
y = ds["cloud_mass_radius_mean"]
x = ds["cloud_liquid_water_content"]

color_data = color_data.sel(microphysics=plot_microphysics)
x = x.sel(microphysics=plot_microphysics)
y = y.sel(microphysics=plot_microphysics)

cmap = strength_cmap


norm = mcolors.Normalize(
    vmin=color_data.compute().quantile(0.1).values, vmax=color_data.compute().quantile(0.9).values
)

for i, mp in enumerate(plot_microphysics):

    # cmap = sns.cubehelix_palette(start=i * 0.2, rot=-0.5, as_cmap=True)

    c = color_data.sel(microphysics=mp)
    ax = axs_flat[i]

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=70,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
save_figure(fig, fig_dir, f"{x.name}-{y.name}-color-{color_data.name}")

### Reconstruct Evaporation from LWC and MMR

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=(7, 5))

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = ds["cloud_mass_radius_mean"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
x = -ds["source_precipitation"]

y = ds["cloud_mass_radius_mean"] ** (-5 / 2) * ds["cloud_liquid_water_content"]
y.attrs.update(long_name="Reconstruction", units="m^{-5/2} * kg / m^3")
y.name = "reconstructed_evaporation"

color_data = color_data.sel(microphysics=plot_microphysics)
x = x.sel(microphysics=plot_microphysics)
y = y.sel(microphysics=plot_microphysics)

cmap = strength_cmap


norm = mcolors.Normalize(
    vmin=color_data.compute().quantile(0.1).values, vmax=color_data.compute().quantile(0.9).values
)

for i, mp in enumerate(plot_microphysics):

    # cmap = sns.cubehelix_palette(start=i * 0.2, rot=-0.5, as_cmap=True)

    c = color_data.sel(microphysics=mp)
    ax = axs_flat[i]

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=70,
        # label = labels[mp],
    )
    # ax.set_yscale("log")
    # ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
save_figure(fig, fig_dir, f"{x.name}-{y.name}-color-{color_data.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ds["microphysics"].values:

    ax.scatter(
        ds["cloud_liquid_water_content"].sel(microphysics=mp),
        -ds["source_precipitation"].sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(ds["cloud_liquid_water_content"]))
ax.set_ylabel(label_from_attrs(ds["source_precipitation"]))

Text(0, 0.5, 'Evaporation $\\left[  mm \\, h^{-1}  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

for mp in ["condensation"]:

    ax.scatter(
        ds["cloud_mass_radius_mean"].sel(microphysics=mp),
        -ds["source_precipitation"].sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(ds["cloud_mass_radius_mean"]))
ax.set_ylabel(label_from_attrs(ds["source_precipitation"]))

Text(0, 0.5, 'Evaporation $\\left[  mm \\, h^{-1}  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = -ds["source_precipitation"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.savefig(fig_dir / f"{x.name}-{y.name}.pdf")

## Interpolation for vertical properties

In [ ]:
np.random.seed(2938)
cloud_ids = np.random.choice(ds["cloud_id"], 5, replace=False)

ds_sel = ds.isel(microphysics=2).sel(cloud_id=cloud_ids).sel(gridbox=[0, 10, 20, 30, 40, 50])
ds_sel = ds_sel.drop_vars("microphysics")


ds_example = xr.Dataset()
da = (ds_sel["massdelta_condensation"] * 0 + ds_sel["gridbox"]).compute()
norm_gridbox = ds_sel["gridbox"] / da.max("gridbox")

ds_example["data"] = da
ds_example["data"].attrs = dict(long_name="Example data", units="kg")
# ds_example['normalized_gridbox'] = xr.where(norm_gridbox <= 1, norm_gridbox, np.nan)
ds_example["normalized_gridbox"] = norm_gridbox
ds_example["normalized_gridbox"].attrs = dict(long_name="Normalized Gridbox", units="")
ds_example = ds_example.chunk({"cloud_id": -1})
ds_example["x"] = ("cloud_id", np.arange(len(ds_example["cloud_id"])))
ds_example = ds_example.swap_dims({"cloud_id": "x"}).drop_vars("cloud_id").transpose("gridbox", "x")
ds_example = ds_example.compute()
ds_example.attrs.update(
    name="example_data",
    description="Example data to test normalization",
)
ds_example.to_netcdf("example_data.nc")

In [ ]:
ds_example = xr.open_dataset("example_data.nc").compute()
display(ds_example)


fig, axs = plt.subplots(ncols=3, width_ratios=(1, 1, 0.1))
c = ds_example["data"]
x = ds_example["x"]

# Plot the data in physical coordinates
y = ds_example["gridbox"]
xx, yy = np.meshgrid(x, y)
sc = axs[0].scatter(xx, yy, c=c, s=100, cmap="inferno_r")
axs[0].set_xlabel(label_from_attrs(x, return_units=False))
axs[0].set_ylabel(label_from_attrs(y, return_units=False))
axs[0].set_title("Data in physical coordinates")


# Plot the data in normalized coordinates
y = ds_example["normalized_gridbox"]
xx, yy = x.expand_dims(gridbox=ds_example["gridbox"]), ds_example["normalized_gridbox"]
sc = axs[1].scatter(xx, yy, c=c, s=100, cmap="inferno_r")

axs[1].set_xlabel(label_from_attrs(x, return_units=False))
axs[1].set_ylabel(label_from_attrs(y, return_units=False))
axs[1].set_title("Data in normalized coordinates")

# Add colorbar
fig.colorbar(sc, cax=axs[2], label="Data")
fig.tight_layout()

In [ ]:
# ds_demo = ds_sel
# key = "mass_moment1"

# fig, axes = plt.subplots(ncols=2, figsize=(10, 4))
# ds_demo[key].plot(ax=axes[0])
# axes[0].set_title("Raw data")
# # Interpolated data
# new_lon = np.linspace(ds_demo['gridbox'].min(), ds_demo['gridbox'].max(), ds_demo.sizes["gridbox"] * 4)
# # new_lat = np.linspace(ds_demo.lat[0].item(), ds_demo.lat[-1].item(), ds_demo.sizes["cloud_id"] * 4)
# ds_demoi = ds_demo.interp(gridbox=new_lon)
# # ds_demoi[key].plot(ax=axes[1])
# # axes[1].set_title("Interpolated data")
# z = ds_demo['cloud_id'].values
# x = ds_demo['gridbox']
# n = ds_demo['max_gridbox']
# N = x / n  / z
# x = np.linspace(N.min(), N.max(), 62)
# Nv = N.values
# # # relation between new and original coordinates
# cloud_id = xr.DataArray(z, dims=["cloud_id"], coords={"cloud_id": z})
# gridbox = xr.DataArray(
#     Nv,
#     dims=["x", "cloud_id"],
#     coords={"x": x, "cloud_id": z},
# )

# fig, axes = plt.subplots(ncols=2, figsize=(10, 4))
# ds_demo[key].plot(ax=axes[0])

# axes[0].set_title("Raw data")


# ds_demoi = ds_demo.interp(gridbox=gridbox, cloud_id=cloud_id)
# ds_demoi
# ds_demoi[key].plot(ax=axes[1])
# axes[1].set_title("Remapped data")

In [ ]:
# ds_demo = xr.tutorial.open_dataset("air_temperature").isel(time=0)
# key = "air"

# fig, axes = plt.subplots(ncols=2, figsize=(10, 4))
# ds_demo[key].plot(ax=axes[0])
# axes[0].set_title("Raw data")
# # Interpolated data
# new_lon = np.linspace(ds_demo.lon[0].item(), ds_demo.lon[-1].item(), ds_demo.sizes["lon"] * 4)
# new_lat = np.linspace(ds_demo.lat[0].item(), ds_demo.lat[-1].item(), ds_demo.sizes["lat"] * 4)
# ds_demoi = ds_demo.interp(lat=new_lat, lon=new_lon)
# ds_demoi[key].plot(ax=axes[1])
# axes[1].set_title("Interpolated data")

# x = np.linspace(240, 300, 100)
# z = np.linspace(20, 70, 100)
# # relation between new and original coordinates
# lat = xr.DataArray(z, dims=["z"], coords={"z": z})
# lon = xr.DataArray(
#     (x[:, np.newaxis] - 270) / np.cos(z * np.pi / 180) + 270,
#     dims=["x", "z"],
#     coords={"x": x, "z": z},
# )

# fig, axes = plt.subplots(ncols=2, figsize=(10, 4))
# ds_demo[key].plot(ax=axes[0])
# # draw the new coordinate on the original coordinates.
# for idx in [0, 33, 66, 99]:
#     axes[0].plot(lon.isel(x=idx), lat, "--k")

# for idx in [0, 33, 66, 99]:
#     axes[0].plot(*xr.broadcast(lon.isel(z=idx), lat.isel(z=idx)), "--k")

# axes[0].set_title("Raw data")


# ds_demoi = ds_demo.interp(lon=lon, lat=lat)

# ds_demoi[key].plot(ax=axes[1])
# axes[1].set_title("Remapped data")